## Import libraries

In [1]:
accesibilidad_urbana = '../../'

In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(accesibilidad_urbana))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Load, count and sumarize data

In [3]:
# Summary dataframe
summary_df = pd.DataFrame()

# Parameters
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'
year = '2020'

# Load all cities
query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")
print(f"Loaded all cities.")

# Iterate over each city
i = 0
city_list = list(metro_gdf.city.unique()) # Full run
#city_list = ['Aguascalientes'] # Test
for city in city_list:

    summary_df.loc[i,'city'] = city # Register data
    
    # 1.1 --------------- EXTRACT CITY DATA FROM METRO_GDF
    # Extract current city
    city_gdf = metro_gdf.loc[metro_gdf.city == city]
    city_gdf = city_gdf.set_crs("EPSG:4326")
    # Create a list with all unique cvegeo_mun ('CVE_ENT'+'CVE_MUN') of current city
    city_gdf['cvegeo_mun'] = city_gdf['CVE_ENT']+city_gdf['CVE_MUN']
    cvegeo_mun_lst = list(city_gdf.cvegeo_mun.unique())
    cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    # To avoid error that happens when there's only one MUN in State: 
    # SQL e.g.: <<< SELECT * FROM censo.censo_inegi_{year[:2]}_mza WHERE ("entidad" = '02') AND "mun" IN ('001',) >>>
    # Duplicate mun inside tupple if there's only one MUN.
    if len(cvegeo_mun_lst) >= 2:
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    else:
        cvegeo_mun_lst.append(cvegeo_mun_lst[0])
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))

    # 1.2 --------------- LOAD AND COUNT POP DATA (AGEBs and Blocks)
    cols = 'pobtot'
    # Load AGEBs and blocks
    query = f"SELECT {cols} FROM censo.censo_inegi_{year[:2]}_ageb WHERE \"cvegeo_mun\" IN {cvegeo_mun_tpl}"
    pop_ageb_gdf = aup.df_from_query(query)
    query = f"SELECT {cols} FROM censo.censo_inegi_{year[:2]}_mza WHERE \"cvegeo_mun\" IN {cvegeo_mun_tpl}"
    pop_mza_gdf = aup.df_from_query(query)
    # Count population
    summary_df.loc[i,'agebs_pop'] = pop_ageb_gdf['pobtot'].sum() # Register data
    summary_df.loc[i,'blocks_pop'] = pop_mza_gdf['pobtot'].sum() # Register data
    print(f"{city} - Loaded AGEBs and blocks.")
    del pop_ageb_gdf
    del pop_mza_gdf

    # 1.3 --------------- LOAD AND COUNT POP DATA (Nodes)
    try:
        cols = 'pobtot'
        # Load nodes
        query = f"SELECT {cols} FROM censo.pobcenso_inegi_{year[:2]}_mzaageb_node WHERE \"city\" LIKE \'{city}\'"
        pop_nodes_gdf = aup.df_from_query(query)
        # Count population
        summary_df.loc[i,'nodes_pop'] = pop_nodes_gdf['pobtot'].sum() # Register data
        print(f"{city} - Loaded nodes.")
        del pop_nodes_gdf
    except:
        print(f"{city} - Has no nodes data.")

    # 1.4 --------------- LOAD AND COUNT POP DATA (Hexs)
    try:
        cols = 'pobtot,dens_pob_ha'
        res_lst = [8,9,10]
        for res in res_lst:
            # Load nodes
            query = f"SELECT {cols} FROM censo.pobcenso_inegi_{year[:2]}_mzaageb_hex WHERE \"city\" = '{city}\' AND \"res\" = '{res}\'"
            pop_hexs_gdf = aup.df_from_query(query)
            # Count population
            summary_df.loc[i,f'hex_{res}_pop'] = pop_hexs_gdf['pobtot'].sum() # Register data
            # Mean density
            summary_df.loc[i,f'hex_{res}_dens'] = pop_hexs_gdf['dens_pob_ha'].mean() # Register data
            print(f"{city} - Loaded hexs res {res}.")
            del pop_hexs_gdf
    except:
        print(f"{city} - Has no hexs data.")

    i+=1

summary_df

Loaded all cities.
Aguascalientes - Loaded AGEBs and blocks.
Aguascalientes - Loaded nodes.
Aguascalientes - Loaded hexs res 8.
Aguascalientes - Loaded hexs res 9.
Aguascalientes - Loaded hexs res 10.
Ensenada - Loaded AGEBs and blocks.
Ensenada - Loaded nodes.
Ensenada - Loaded hexs res 8.
Ensenada - Loaded hexs res 9.
Ensenada - Loaded hexs res 10.
Mexicali - Loaded AGEBs and blocks.
Mexicali - Loaded nodes.
Mexicali - Loaded hexs res 8.
Mexicali - Loaded hexs res 9.
Mexicali - Loaded hexs res 10.
Tijuana - Loaded AGEBs and blocks.
Tijuana - Loaded nodes.
Tijuana - Loaded hexs res 8.
Tijuana - Loaded hexs res 9.
Tijuana - Loaded hexs res 10.
La Paz - Loaded AGEBs and blocks.
La Paz - Loaded nodes.
La Paz - Loaded hexs res 8.
La Paz - Loaded hexs res 9.
La Paz - Loaded hexs res 10.
Los Cabos - Loaded AGEBs and blocks.
Los Cabos - Loaded nodes.
Los Cabos - Loaded hexs res 8.
Los Cabos - Loaded hexs res 9.
Los Cabos - Loaded hexs res 10.
Campeche - Loaded AGEBs and blocks.
Campeche - Lo

Exception during reset or similar
Traceback (most recent call last):
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 145, in __init__
    self._dbapi_connection = engine.raw_connection()
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3292, in raw_connection
    return self.pool.connect()
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 452, in connect
    return _ConnectionFairy._checkout(self)
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 1269, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 716, in checkout
    rec = pool._do_get()
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/pool/impl.py", line 170, in _do_get
    self._dec_overflow()
  File "/opt/conda/envs/gds/lib/python3.9/site-packages/sqlalchemy/util/l

OperationalError: (psycopg2.OperationalError) connection to server at "hippocampus.cswst4rid7eb.us-east-2.rds.amazonaws.com" (18.190.20.157), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "hippocampus.cswst4rid7eb.us-east-2.rds.amazonaws.com" (18.190.20.157), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [9]:
summary_df['blocks_agebs'] = summary_df['blocks_pop']-summary_df['agebs_pop']

summary_df['blocks_nodes'] = summary_df['blocks_pop']-summary_df['nodes_pop']
summary_df['blocks_hex8'] = summary_df['blocks_pop']-summary_df['hex_8_pop']
summary_df['blocks_hex9'] = summary_df['blocks_pop']-summary_df['hex_9_pop']
summary_df['blocks_hex10'] = summary_df['blocks_pop']-summary_df['hex_10_pop']

summary_df.head(40)

,city,agebs_pop,blocks_pop,nodes_pop,hex_8_pop,hex_8_dens,hex_9_pop,hex_9_dens,hex_10_pop,hex_10_dens,blocks_nodes,blocks_hex8,blocks_hex9,blocks_hex10,blocks_agebs
0,Aguascalientes,1042295.0,1043905.0,1.043905e+06,1.043905e+06,38.002939,1.043905e+06,51.645011,1.043905e+06,71.190394,2.348462e-02,1.904223e-02,2.606417e-02,2.403932e-02,1610.0
1,Ensenada,397099.0,396709.0,3.967090e+05,3.967090e+05,20.444583,3.967090e+05,28.672940,3.967090e+05,42.307928,-1.736394e-02,-1.632396e-02,-1.747433e-02,-1.688840e-02,-390.0
2,Mexicali,950970.0,959257.0,9.592567e+05,9.592567e+05,22.453421,9.592567e+05,31.651314,9.592567e+05,46.078042,2.740791e-01,2.705197e-01,2.694871e-01,2.741234e-01,8287.0
3,Tijuana,2000259.0,2002469.0,2.002469e+06,2.002469e+06,41.885156,2.002469e+06,50.672541,2.002469e+06,64.161687,-2.957222e-02,-3.127077e-02,-2.867546e-02,-2.929308e-02,2210.0
4,La Paz,270846.0,270681.0,2.706810e+05,2.706810e+05,13.663785,2.706810e+05,20.864971,2.706810e+05,34.546294,6.533141e-05,4.495538e-03,2.852159e-04,6.789260e-05,-165.0
5,Los Cabos,338979.0,338675.0,3.386750e+05,3.386750e+05,16.880406,3.386750e+05,24.377736,3.386750e+05,39.935697,-2.491545e-03,-6.585740e-03,-9.013808e-04,-2.633244e-03,-304.0
6,Campeche,267558.0,267387.0,2.673870e+05,2.673870e+05,29.426850,2.673870e+05,41.549685,2.673870e+05,59.098960,1.649003e-02,1.771487e-02,1.786993e-02,1.658175e-02,-171.0
7,Laguna,1218759.0,1218686.0,1.218686e+06,1.218686e+06,29.634965,1.218686e+06,39.417845,1.218686e+06,55.162776,7.384770e-02,7.250686e-02,7.437077e-02,7.303434e-02,-73.0
8,Monclova,364881.0,367434.0,3.674340e+05,3.674340e+05,19.238868,3.674340e+05,26.573681,3.674340e+05,38.978046,-5.825941e-03,-6.369926e-03,-6.783308e-03,-5.574614e-03,2553.0
9,Piedras Negras,206172.0,206163.0,2.061630e+05,2.061630e+05,17.494641,2.061630e+05,24.315004,2.061630e+05,34.612875,-4.939427e-04,-1.684350e-03,-4.042810e-04,-1.656910e-04,-9.0


In [13]:
summary_df.tail(26)

,city,agebs_pop,blocks_pop,nodes_pop,hex_8_pop,hex_8_dens,hex_9_pop,hex_9_dens,hex_10_pop,hex_10_dens,blocks_nodes,blocks_hex8,blocks_hex9,blocks_hex10,blocks_agebs
40,Puebla,2654156.0,2647940.0,2.647940e+06,2.647940e+06,31.899048,2.647940e+06,39.583086,2.647940e+06,54.441507,4.079699e-02,3.348014e-02,3.904257e-02,4.077408e-02,-6216.0
41,San Martin,287426.0,285397.0,2.853970e+05,2.853970e+05,14.858234,2.853970e+05,22.343119,2.853970e+05,38.954284,-1.413995e-02,-1.330954e-02,-1.280071e-02,-1.425192e-02,-2029.0
42,Tehuacan,338779.0,337788.0,3.377880e+05,3.377880e+05,25.126734,3.377880e+05,37.541128,3.377880e+05,56.776551,-2.002805e-02,-2.290040e-02,-1.916919e-02,-2.005740e-02,-991.0
43,Queretaro,1381392.0,1379350.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,1.379350e+06,1.379350e+06,1.379350e+06,1.379350e+06,-2042.0
44,Cancun,928428.0,928339.0,9.283391e+05,9.283391e+05,40.099181,9.283391e+05,53.059133,9.283391e+05,73.346812,-1.120071e-01,-1.180139e-01,-1.128181e-01,-1.116748e-01,-89.0
45,Chetumal,188996.0,188777.0,1.887770e+05,1.887770e+05,20.374186,1.887770e+05,30.267272,1.887770e+05,47.666899,4.362037e-03,1.793865e-03,4.704250e-03,4.442726e-03,-219.0
46,Playa,327820.0,327652.0,3.276520e+05,3.276520e+05,38.590589,3.276520e+05,51.538820,3.276520e+05,76.802945,2.670948e-02,2.864301e-02,2.495929e-02,2.605055e-02,-168.0
47,SLP,1194744.0,1194500.0,1.194500e+06,1.194500e+06,39.201861,1.194500e+06,50.579772,1.194500e+06,68.069126,-1.601504e-02,-1.691211e-02,-1.694176e-02,-1.745588e-02,-244.0
48,Culiacan,886283.0,886053.0,8.860531e+05,8.860531e+05,33.187409,8.860531e+05,44.954043,8.860531e+05,60.650179,-6.967195e-02,-7.248399e-02,-6.821082e-02,-6.960061e-02,-230.0
49,Los Mochis,378508.0,380961.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,3.809610e+05,3.809610e+05,3.809610e+05,3.809610e+05,2453.0


In [21]:
local_save_dir = accesibilidad_urbana + 'data/processed/pop_data/pop_data_comparison.csv'
save = True
if save:
    summary_df.to_csv(local_save_dir)